In [1]:
%cd ..

C:\Users\13661\PycharmProjects\KinshipVerification
C:\Users\13661\PycharmProjects\KinshipVerification


In [2]:
import numpy as np
import torch
import torchvision
from torch.utils.data.dataloader import DataLoader

device = torch.device('cuda')

In [3]:
from dataset import kinfaceW

In [4]:
train_split = np.load('kfw_train_pair.npy')
dataset = kinfaceW(train_split)
loader = DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1)

In [5]:
from facenet_pytorch import InceptionResnetV1
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [6]:
from tqdm import tqdm
import torch.nn.functional as F

outs = []
with torch.no_grad():
    with tqdm(loader, unit="batch") as data_loader:
        for batch in data_loader:
            label = int(batch[2][0])
            batch = [v.permute(0,3,1,2).to(device) for v in batch[:2]]

            p = resnet(batch[0]).squeeze().detach().cpu()
            c = resnet(batch[1]).squeeze().detach().cpu()
            
            outs.append([F.normalize(torch.abs(p - c), dim=0).numpy(), label])

100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [05:18<00:00,  6.27batch/s]



In [7]:
import random
random.shuffle(outs)

X = [a[0] for a in outs]
Y = [a[1] for a in outs]

In [8]:
print(len(X))

2000
2000


In [11]:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score

clf = svm.SVC()

clf.fit(X,Y)
clf.score(X,Y)

0.8895

0.8895

In [12]:
test_split =  np.load('kfw_test_pair.npy')
testset = kinfaceW(test_split)
testloader = DataLoader(
        testset, batch_size=1, shuffle=False, num_workers=1)

In [13]:
from tqdm import tqdm
import torch.nn.functional as F

t_outs = []
with torch.no_grad():
    with tqdm(testloader, unit="batch") as data_loader:
        for batch in data_loader:
            label = int(batch[2][0])
            batch = [v.permute(0,3,1,2).to(device) for v in batch[:2]]

            p = resnet(batch[0]).squeeze().detach().cpu()
            c = resnet(batch[1]).squeeze().detach().cpu()
            
            t_outs.append([F.normalize(torch.abs(p - c), dim=0).numpy(), label])

100%|███████████████████████████████████████████████████████████████████████████| 1066/1066 [02:49<00:00,  6.27batch/s]



In [14]:
random.shuffle(t_outs)

test_x = [a[0] for a in t_outs]
test_y = [a[1] for a in t_outs]

In [15]:
print(len(test_x))

1066
1066


In [18]:
clf.score(test_x, test_y)

0.5797373358348968

0.5797373358348968